In [8]:
import faiss
import re
from langchain_community.chat_models import ChatOllama
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter , RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [9]:
#loader = PyPDFLoader("자서전 가상 100문100답.pdf")
#documents = loader.load()
loaders = [
            PyPDFLoader('자서전 가상 100문100답.pdf'),
            PyPDFLoader('개인이력서.pdf')
        ]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=100)
all_splits = []

for loader in loaders:
    pages = loader.load_and_split()
    splits = text_splitter.split_documents(pages)
    all_splits.extend(splits)


Ignoring wrong pointing object 6 0 (offset 0)


In [10]:
model_name = "jhgan/ko-sroberta-multitask"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

In [11]:
db = Chroma.from_documents(splits, embeddings)

In [12]:
llm = ChatOllama(model="ggml-model-Q5_K_M/Modelfile:latest")

In [14]:
system_template = """당신은 자서전 작가입니다. 
생전에 사용자가 제공한 정보를 바탕으로 자서전을 풍부하고 감동적으로 구성해야 합니다.
다음의 질문에 대한 답변을 제공할 때, 사용자의 삶의 경험과 이야기를 고려하여 깊이 있는 답변을 생성하십시오.
답변 분량은 최소 500자 이상입니다.
PDF 문서에 없는 내용은 답변하지 않습니다.
{context}
"""

In [15]:
human_template = "{question}"

In [16]:
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template)
])

In [17]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs={"prompt": chat_prompt}
)

In [18]:
def ask_question(question):
    return qa_chain.run(question)

In [19]:
question = "사용자가 제공한 정보를 바탕으로 자서전을 원하는데 6가지 챕터 제목은 뭐가 좋을까요?"
context = ask_question(question)
print(f"질문: {question}")
print(f"답변: {context}")

C:\Users\emine\AppData\Local\Temp\ipykernel_25232\1326970588.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  return qa_chain.run(question)


질문: 사용자가 제공한 정보를 바탕으로 자서전을 원하는데 6가지 챕터 제목은 뭐가 좋을까요?
답변: 제목: \_\_\_\_의 여정: 용기, 결단력, 그리고 꿈 실현에 대한 이야기\_\_\_\_

1장: 새로운 시작: 성장과 배움의 초기 시절
이 장에서는 사용자의 초창기 생애를 다룹니다. 가족 배경, 성장 환경, 유년 시절 경험 등을 포함하여 이러한 경험들이 그들의 성격을 형성하는 데 어떤 역할을 했는지 설명합니다. 또한 교육 여정을 탐구하며 그들이 어떻게 학습에 대한 열정과 지식을 추구하는 것을 발견했는지 보여줍니다.

2장: 꿈을 향한 한 걸음: 경력의 시작과 도전 극복하기
이 장에서는 사용자의 초기 경력 발판을 다룹니다. 첫 직책, 겪었던 장애물들, 그리고 이 시기에 형성된 중요한 관계 등을 자세히 설명합니다. 또한 사용자가 직면했던 어려움들을 어떻게 극복하고 이러한 경험들이 성장과 전문성 개발에 어떤 역할을 했는지를 논의합니다.

3장: 변곡점: 새로운 시작과 인생 변화의 기회
변곡점은 사용자의 삶에서 중요한 전환점을 강조합니다. 그들이 경력, 개인 생활 또는 가치관에 있어 큰 변화를 겪었을 때를 이야기합니다. 이 결정이 그들의 정체성을 형성하고 미래의 행동을 안내하는 데 어떤 역할을 했는지 설명합니다.

4장: 성취와 인정: 성공의 순간들
성공과 인정을 통해 사용자의 성취를 축하하는 장입니다. 그들이 일생 동안 이룬 중요한 성과들과 이러한 업적이 개인적인 삶뿐만 아니라 커뮤니티에 미친 영향을 강조합니다. 또한 이러한 성공을 이루기 위해 기울인 노력과 헌신을 인정합니다.

5장: 회복력과 성장: 역경, 그리고 그것들을 극복하기 위한 여정
이 장은 사용자가 직면했던 도전과 장애물을 다루며, 이들이 그들의 인격 형성에 어떤 역할을 했는지를 강조합니다. 그들이 고난을 어떻게 극복하고 역경을 경험으로 전환했는지에 대해 이야기하며, 이러한 경험들이 그들을 더욱 강하고 회복력 있게 만들었음을 보여줍니다.

6장: 미래를 향한 비전: 여정에서의 교훈과 얻은 통찰
이 

In [26]:
system_template1 = """ 
1장: 새로운 시작: 성장과 배움의 초기 시절
이 장에서는 사용자의 초창기 생애를 다룹니다. 가족 배경, 성장 환경, 유년 시절 경험 등을 포함하여 이러한 경험들이 그들의 성격을 형성하는 데 어떤 역할을 했는지 설명합니다. 또한 교육 여정을 탐구하며 그들이 어떻게 학습에 대한 열정과 지식을 추구하는 것을 발견했는지 보여줍니다.
참조된 PDF 문서에 인물, 장소, 취미, 직업, 전공 들을 생성하지 않습니다. 첨부된 PDF에 내용만 가지고 작성합니다.
{context}
"""

In [27]:
human_template1 = "{question}"

In [28]:
chat_prompt1 = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template1),
    HumanMessagePromptTemplate.from_template(human_template1)
])

In [29]:
qa_chain1 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs={"prompt": chat_prompt1}
)

In [30]:
def ask_question1(question):
    return qa_chain1.run(question)

In [31]:
question = "1장: 새로운 시작: 성장과 배움의 초기 시절을 주제로 작성해주세요"
context = ask_question1(question)
print(f"질문: {question}")
print(f"답변: {context}")

질문: 1장: 새로운 시작: 성장과 배움의 초기 시절을 주제로 작성해주세요
답변: **새로운 시작: 성장과 배움의 초기 시절**

초기 생애와 가족 배경
저는 [도시]에서 태어나고 자라며, 부모님인 [부모님 이름]의 사랑과 지지 속에서 성장하였습니다. 부모님은 항상 제가 열심히 일하고 성실함을 중시하는 태도를 가지도록 격려해주셨습니다. 저희 아버지는 헌신적인 엔지니어이시고 어머니는 학교 선생님으로 일하시며 교육에 대한 열정을 가지고 계셨습니다.

저의 유년 시절은 언제나 따뜻하고 포근한 분위기 속에서 이루어졌습니다. 저는 두 명의 남동생과 함께 자라며, 형제자매들과 친밀하고 지지하는 관계를 유지했습니다. 저희 가족은 항상 가까운 유대감을 유지하며, 서로의 성취를 자랑스러워하고 어려움에 직면했을 때 서로를 도왔습니다.

교육 여정
저는 어린 시절부터 학습에 대한 열정을 가지고 있었습니다. 부모님 모두 교육을 매우 중시하셨고, 항상 독서, 글쓰기 그리고 지적 호기심을 장려하셨습니다. 저는 초등학교와 중학교에서 뛰어난 학생으로 활발한 활동 참가자이자 학급 리더로 활약했습니다.

저는 명문 고등학교인 [학교 이름]에 진학하여 우수한 성적으로 졸업하였습니다. 고등학교 재학 시절, 다양한 과목을 수강하며 제 열정을 발견하는 데 시간을 보냈습니다. 과학과 수학에 대한 자연스러운 재능을 발휘해 물리학과 수학을 전공으로 선택하게 되었습니다.

성취와 도전
고등학교 시절 동안 저는 학문적 우수성을 인정받아 다수의 상을 수상했습니다. [수상 내역] 등입니다. 학업 성취 외에도 스포츠, 음악, 사회 봉사 활동에 적극적으로 참여하여 팀워크, 리더십, 그리고 책임감을 배웠습니다.

학업 여정 내내 몇 가지 도전을 마주하기도 했습니다. 특히 고등학교 때 저는 대학 진학 준비 과정과 도전적인 과목들을 균형 있게 관리하는 데에 어려움을 겪었습니다. 하지만 이러한 도전에 맞서며 배우고 성장할 수 있었으며 결국에는 넘어서게 되었습니다.

초기 사회생활
처음 사회생활을 시작했을 때는 새로운 환경

In [103]:
system_template2 = """ PDF 문서를 참고하여 유년시절을 주제로 작성합니다.
2장: 배움의 여정: 지평선의 향연
이 챕터에서 사용자는 교육 여정을 자세히 다루며 학문적 업적, 도전 과제, 그리고 중요한 교수님들과 멘토들과의 만남을 강조합니다. 그들은 글쓰기에 대한 열정과 창의성에 대한 사랑을 키워준 다양한 과외 활동과 경험에 대해서도 이야기할 것입니다.
가상의 인물,장소,취미,직업,전공 들을 생성하지 않습니다. PDF에 내용만 가지고 작성합니다.
{context}
"""

In [104]:
human_template2 = "{question}"

In [105]:
chat_prompt2 = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template2),
    HumanMessagePromptTemplate.from_template(human_template2)
])

In [106]:
qa_chain2 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs={"prompt": chat_prompt2}
)

In [107]:
def ask_question2(question):
    return qa_chain2.run(question)

In [108]:
question = "배움의여정을 주제로 작성해주세요"
context = ask_question2(question)
print(f"질문: {question}")
print(f"답변: {context}")

질문: 배움의여정을 주제로 작성해주세요
답변: 2장: 배움의 여정: 지평선의 향연
유년시절부터 교육에 대한 사랑과 지식이 가져다주는 무한한 가능성에 매료되었습니다. 어린 시절부터 저는 호기심 많은 학생이었고, 배우는 것을 정말 좋아했습니다. 새로운 개념을 접하고 지식을 탐구하는 것은 저에게 매우 신나고 보람찬 일이었습니다.
제 학습 여정은 학부 때부터 시작되어 명문대학교에서 영어와 철학을 전공으로 학사 학위를 받았습니다. 이곳에서 저는 다양한 학문적 경험을 할 기회를 가졌으며, 창의성과 글쓰기 기술을 연마할 수 있었습니다. 또한 제 관심사를 탐구하고 새로운 것을 배울 수 있는 여러 과외 활동에 참여했습니다.
학부 이후에는 저널리즘 분야에 진출하여 명성 있는 잡지사와 신문사에서 기자로 일하면서 경력을 시작했습니다. 여기에서는 다양한 주제와 취재를 다뤘으며, 이는 글쓰기 및 소통 기술을 강화하는 데 도움이 되었습니다. 또한 기사 작성과 편집에 필요한 기술과 전문성을 개발하는 데에도 도움이 되었습니다.
경력을 이어가면서 저는 여러 교육 기관에서 글쓰기와 커뮤니케이션을 가르치는 기회를 가졌습니다. 이러한 경험은 학생들에게 지식과 제 열정을 나누는 데 소중한 통찰력과 기술을 제공했습니다.
저의 학문적 여정은 저널리즘, 글쓰기, 의사소통에 대한 강한 헌신으로 특징 지어집니다. 이 분야에서 일하면서 저는 다양한 기술과 지식을 개발했고, 이러한 지식을 다른 사람들과 나누기를 열망하고 있습니다. 교육자로서 저는 학생들에게 창의성과 글쓰기에 대한 열정을 기르고 싶습니다. 또한 그들이 탐구하고 새로운 것을 배울 수 있는 안전한 공간을 만들고자 합니다.
저는 교육의 힘을 믿으며 지식이 세상을 변화시킬 수 있다고 생각합니다. 학생들이 자신의 잠재력을 최대한 발휘할 수 있도록 지원하는 데 헌신하며, 제 경험과 지식을 공유하여 그들에게 지식과 창의성에 대한 사랑을 심어주고 싶습니다.
